In [ ]:
import tensorflow as tf
import cv2
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from keras.utils import to_categorical
from keras.layers import Dropout, Flatten,Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

In [ ]:
import os
import zipfile


In [ ]:
# drive.mount('/content/drive/')
 
zip_ref = zipfile.ZipFile("/content/drive/My Drive/flowers_dataset.zip", 'r')
zip_ref.extractall("/tmp/flowers_dataset")
zip_ref.close()

In [ ]:

X=[]
Z=[]
IMG_SIZE=200
daisy_dir='/tmp/flowers_dataset/flowers/daisy'
sunflower_dir='/tmp/flowers_dataset/flowers/sunflower'
tulip_dir='/tmp/flowers_dataset/flowers/tulip'
dandelion_dir='/tmp/flowers_dataset/flowers/dandelion'
rose_dir='/tmp/flowers_dataset/flowers/rose'

In [ ]:
def assign_label(img,flower_type):
    return flower_type
    
def make_train_data(flower_type,DIR):
      for img in tqdm(os.listdir(DIR)):
        _, ftype = os.path.splitext(img)
        if ftype == ".jpg": 
          label=assign_label(img,flower_type)
          path = os.path.join(DIR,img)
          img = cv2.imread(path)
          img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
          
          X.append(np.array(img))
          Z.append(str(label))

In [ ]:
make_train_data('Daisy',daisy_dir)
make_train_data('Sunflower',sunflower_dir)
make_train_data('Tulip',tulip_dir)
make_train_data('Rose',rose_dir)
make_train_data('Dandelion',dandelion_dir)


100%|██████████| 1055/1055 [00:03<00:00, 344.35it/s]


In [ ]:
le=LabelEncoder()
Y=le.fit_transform(Z)
Y=to_categorical(Y,5)
X=np.array(X)
X=X/255

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.19,random_state=42)


In [ ]:
datagen = ImageDataGenerator(
         rescale=1/255)  

# datagen.fit(x_train)

train_generator = datagen.flow_from_directory(
        '/tmp/flowers_dataset/flowers/',  # This is the source directory for training images
        target_size=(200, 200),  # All images will be resized to 150x150
        batch_size=128,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='categorical')

Found 4323 images belonging to 5 classes.


In [ ]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 200x200 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(32, (5,5), activation='relu', padding='same', input_shape=(200, 200, 3)),
    # tf.keras.layers.Conv2D(96, (5,5), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(96, (3,3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(2,2),
    # # The fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(16, (3,3), activation='relu'),
    # tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(288, activation='relu'),
    # tf.keras.layers.Dropout(0.4),
    # tf.keras.layers.Dense(512, activation='relu'),
    # tf.keras.layers.Dropout(0.4),
    # tf.keras.layers.Dense(128, activation='relu'),
    # tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(5, activation='softmax')
    ])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 200, 200, 32)      2432      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 100, 100, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 100, 100, 96)      27744     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 50, 96)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 50, 50, 64)        55360     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 25, 25, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 25, 25, 64)        3

In [ ]:
model.compile(optimizer=RMSprop(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
batch_size=128
epochs=60
History = model.fit(train_generator,
                              epochs = epochs, validation_data = (x_test,y_test),
                              verbose = 1, steps_per_epoch=8)

Epoch 1/60
8/8 [==============================] - 127s 16s/step - loss: 1.6702 - accuracy: 0.2119 - val_loss: 1.6064 - val_accuracy: 0.2263
Epoch 2/60
8/8 [==============================] - 124s 16s/step - loss: 1.6047 - accuracy: 0.2185 - val_loss: 1.6839 - val_accuracy: 0.1946
Epoch 3/60
8/8 [==============================] - 123s 16s/step - loss: 1.7362 - accuracy: 0.2130 - val_loss: 1.7907 - val_accuracy: 0.2311
Epoch 4/60
8/8 [==============================] - 123s 16s/step - loss: 1.5630 - accuracy: 0.3124 - val_loss: 1.5982 - val_accuracy: 0.2299
Epoch 5/60
8/8 [==============================] - 124s 16s/step - loss: 1.5135 - accuracy: 0.2931 - val_loss: 1.5492 - val_accuracy: 0.3029
Epoch 6/60
8/8 [==============================] - 125s 16s/step - loss: 1.6585 - accuracy: 0.3096 - val_loss: 1.5904 - val_accuracy: 0.2238
Epoch 7/60
8/8 [==============================] - 124s 16s/step - loss: 1.4689 - accuracy: 0.3519 - val_loss: 1.6448 - val_accuracy: 0.3114
Epoch 8/60
8/8 [====